In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')

In [33]:
from scipy.sparse import coo_matrix,csr_matrix,dok_matrix

In [3]:
#플레이 리스트
train = pd.read_json('data/train.json')
train['tag__'] = train['tags'].apply(lambda x: (" ").join(x))


#곡 정보
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name_basket__'] = song_meta['artist_name_basket'].apply(lambda x: (" ").join(x))
song_meta['song_gn_gnr_basket__'] = song_meta['song_gn_gnr_basket'].apply(lambda x: (" ").join(x))

In [3]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date,tag__
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,록메탈 밴드사운드 록 락메탈 메탈 락 extreme
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,일렉
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,담시 가족 눈물 그리움 주인공 나의_이야기 사랑 친구
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,잔잔한 버스 퇴근버스 Pop 풍경 퇴근길


In [123]:
#텍스트 정제 (특수기호 제거)
import re

txt = []

for i, document in enumerate(train['plylst_title']):
    document = re.sub(r'[^ A-Za-z0-9가-힣]', '', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip()) # 문장 왼쪽 오른쪽에 있는 공백 제거
    
train['playlist_title_txt'] = txt
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tag__,playlist_title_txt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락,여행같은 음악
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상,요즘 너 말야
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한,편하게 잔잔하게 들을 수 있는 곡
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,크리스마스 분위기에 흠뻑 취하고 싶을때
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스,추억의 노래


In [126]:
playlist_title_token = [okt.morphs(x) for x in train['playlist_title_txt']]
playlist_title_token

[['여행', '같은', '음악'],
 ['요즘', '너', '말', '야'],
 ['편하게', '잔잔하게', '들을', '수', '있는', '곡'],
 ['크리스마스', '분위기', '에', '흠뻑', '취하', '고', '싶을', '때'],
 ['추억', '의', '노래'],
 ['2017', 'Pop', 'Trend'],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '을', '자극', '하는', '곡', '들'],
 ['멍청이', '내', '맘', '도', '몰라'],
 ['DANCING', 'IN', 'THE', 'MOONLIGHT', '01'],
 ['록', '메탈', 'Written', 'by', '이일우'],
 ['걸그룹', '땐쓰쏭'],
 ['노래', '로', '의지', '를', '불', '태우자', '1일', '1', '다짐', 'st', '용', '프로필', '뮤직'],
 ['지친', '너', '를', '위로', '해줄', '제목', '편지', '준비', '해봤어'],
 ['트렌디', '하고', '그루브', '한', '힙합', '알앤비', 'MUSIC'],
 ['Autumn', 'in', 'Jazz'],
 ['걍', '게임', '할', '때', '듣는', '음악'],
 ['나', '만', '알고싶은', '노래', '들'],
 ['축가', '듀엣'],
 ['이렇게', '비', '내리는', '날이면', '너', '도', '내', '생각', '할까'],
 ['올림픽', '스밍', '목록'],
 ['조용히', '맥주', '한잔', '할', '때', '새', '벽감', '성'],
 [],
 ['추억', '의', '명화', '를', '만나는', '시간', '영화', 'OST', '모음'],
 ['기분', '좋은', '여름날'],
 ['지친', '하루', '끝', '힐링', '이', '필요한', '당신', '에게', '추천', '하는', '인디', '곡'],
 ['새벽', '찬', '바람', '이', '침대', '위', '를', 

In [119]:
playlist_token = [x.split() for x in txt]
playlist_token_len = [len(x) for x in playlist_token]

In [60]:
# 불용어
stopwords = pd.read_csv('data/stopword.csv')
stopwords = stopwords.iloc[:,0].tolist()
stopwords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를',
 '들면',
 '들자면',
 '저',
 '소인',
 '소생',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수',
 '없다',
 '해서는',
 '안된다',
 '뿐만',
 '아니라',
 '만이',
 '아니다',
 '만은',
 '막론하고',
 '관계없이',
 '그치지',
 '않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지',
 '따지지',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만',
 '못하다',
 '하는',
 '편이',
 '낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이',
 '외에',
 '밖에',
 '하여야',
 '비로소',
 '한다면',
 '몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할',
 '생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게',
 '함으로써',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에',
 '틀림없다',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간'

---
---

In [34]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [7]:
txt.remove('')
txt = list(map(str,txt))
txt

['여행같은 음악',
 '요즘 너 말야',
 '편하게 잔잔하게 들을 수 있는 곡',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래',
 '2017 Pop Trend',
 '짝사랑고백사랑이별슬픔 감성을 자극하는곡들',
 '멍청이 내맘도 몰라',
 'DANCING IN THE MOONLIGHT 01',
 '록메탈 Written by 이일우',
 '걸그룹 땐쓰쏭',
 '노래로 의지를 불태우자 1일1다짐 st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면 너도 내 생각 할까',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때새벽감성',
 '추억의 명화를 만나는 시간 영화 OST 모음',
 '기분좋은 여름날',
 '지친 하루 끝 힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때국외',
 '기분좋은 햇살같은 숨겨진 명곡들',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지 미국 대표 클래식',
 '겨울이 기다리며 미리 들어요',
 '아침이 상쾌해지는 빅밴드의 스윙스윙',
 '여행가는 길 이런음악과 함께 고고씽',
 '믿고듣는 하우스 일렉트로니카 POP 2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자',
 '발라드로 감성을 발라드림슬픔주의',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '890년대 우리나라 라디오에서 흘러나오던 명곡들',
 '영화 Jazz를 만나다 294',
 '우울하고 시크하',
 '극도로 우울할때',
 '폭팔적인 밴드 ROCK 모음집',
 '금쪽같은 휴일 혼자만의 시간을 방해하지 말아요 2',
 'Electronicaloung

In [8]:
token = []
for i in txt:
    token.append(word_tokenize(i))

In [9]:
token

[['여행같은', '음악'],
 ['요즘', '너', '말야'],
 ['편하게', '잔잔하게', '들을', '수', '있는', '곡'],
 ['크리스마스', '분위기에', '흠뻑', '취하고', '싶을때'],
 ['추억의', '노래'],
 ['2017', 'Pop', 'Trend'],
 ['짝사랑고백사랑이별슬픔', '감성을', '자극하는곡들'],
 ['멍청이', '내맘도', '몰라'],
 ['DANCING', 'IN', 'THE', 'MOONLIGHT', '01'],
 ['록메탈', 'Written', 'by', '이일우'],
 ['걸그룹', '땐쓰쏭'],
 ['노래로', '의지를', '불태우자', '1일1다짐', 'st용', '프로필뮤직'],
 ['지친', '너를', '위로해줄', '제목편지', '준비해봤어'],
 ['트렌디하고', '그루브한', '힙합알앤비', 'MUSIC'],
 ['Autumn', 'in', 'Jazz'],
 ['걍게임할때듣는음악'],
 ['나만', '알고싶은', '노래들'],
 ['축가듀엣'],
 ['이렇게', '비', '내리는', '날이면', '너도', '내', '생각', '할까'],
 ['올림픽', '스밍', '목록'],
 ['조용히', '맥주', '한잔', '할때새벽감성'],
 ['추억의', '명화를', '만나는', '시간', '영화', 'OST', '모음'],
 ['기분좋은', '여름날'],
 ['지친', '하루', '끝', '힐링이', '필요한', '당신에게', '추천하는', '인디곡'],
 ['새벽', '찬바람이', '침대위를', '감쌀때국외'],
 ['기분좋은', '햇살같은', '숨겨진', '명곡들'],
 ['사랑스러운', '남녀듀엣'],
 ['즐겨듣는소래'],
 ['닥힙추', '8월', 'Playlist'],
 ['거슈윈에서', '스타워즈까지', '미국', '대표', '클래식'],
 ['겨울이', '기다리며', '미리', '들어요'],
 ['아침이', '상쾌해지는', '빅밴드의', '스윙스윙'],
 ['여행가는', '길',

In [35]:
from konlpy.tag import Okt

In [31]:
okt = Okt()

In [32]:
txt[33]

'믿고듣는 하우스 일렉트로니카 POP 2'

In [28]:
display(okt.morphs(txt[33]))

display(okt.morphs(txt[33], norm = True)) # 문장을 정규화 함 / 엄청 쓰지는 않을듯

display(okt.morphs(txt[33], stem = True)) # 형태소 단위로 나눈 후 어간을 추출

['믿고', '듣는', '하우스', '일렉트로니카', 'POP', '2']

['믿다', '듣다', '하우스', '일렉트로니카', 'POP', '2']

['믿고', '듣는', '하우스', '일렉트로니카', 'POP', '2']

In [30]:
display(okt.nouns(txt[33])) # 명사만 추출
display(okt.phrases(txt[33])) # 텍스트에서 어절만 추출

['하우스', '일렉트로니카']

['하우스', '하우스 일렉트로니카', '하우스 일렉트로니카 POP', '하우스 일렉트로니카 POP 2', '일렉트로니카', 'POP']

In [68]:
display(okt.pos(txt[33])) # 형태소와 품사 추출

display(okt.pos(txt[33], stem = True)) # 형태소의 어간 추출

display(okt.pos(txt[33], stem = True, join =True)) # 형태소와 품사를 붙여서 리스트화 

[('여행가', 'Noun'),
 ('는', 'Josa'),
 ('길', 'Noun'),
 ('이런', 'Modifier'),
 ('음악', 'Noun'),
 ('과', 'Josa'),
 ('함께', 'Adverb'),
 ('고고씽', 'Noun')]

[('여행가', 'Noun'),
 ('는', 'Josa'),
 ('길', 'Noun'),
 ('이런', 'Modifier'),
 ('음악', 'Noun'),
 ('과', 'Josa'),
 ('함께', 'Adverb'),
 ('고고씽', 'Noun')]

['여행가/Noun',
 '는/Josa',
 '길/Noun',
 '이런/Modifier',
 '음악/Noun',
 '과/Josa',
 '함께/Adverb',
 '고고씽/Noun']

---
### TF_IDF 유사도

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [86]:
# TF_IDF를 통한 유사도 구하기
# 벡터화 없이 바로 유사도 구하기 가능함
tfid_vect = TfidfVectorizer()
tfid_matrix = tfid_vect.fit_transform(train['playlist_title_txt'].tolist()) 

In [87]:
idf = tfid_vect.idf_ # 
print(dict(zip(tfid_vect.get_feature_names(),idf)))

{'00': 10.088363940929701, '000': 11.554701009723129, '0000': 11.043875385957136, '00000': 11.960166117831292, '00000600': 11.960166117831292, '0000am당신': 11.960166117831292, '0001': 11.960166117831292, '0002': 11.960166117831292, '0003': 11.960166117831292, '0004': 11.554701009723129, '0004년': 11.960166117831292, '0005': 11.960166117831292, '0007': 11.960166117831292, '0008': 11.960166117831292, '0009': 11.960166117831292, '001': 10.350728205397191, '0010': 11.960166117831292, '0010감성': 11.960166117831292, '002': 11.267018937271347, '003': 11.960166117831292, '005': 11.960166117831292, '0050': 11.960166117831292, '006': 11.554701009723129, '007': 10.861553829163183, '00party': 11.960166117831292, '00s': 10.707403149335924, '00년': 11.267018937271347, '00년대': 9.945263097289027, '00년대초': 11.960166117831292, '00년도': 11.960166117831292, '00년생': 11.960166117831292, '00년생부터': 11.960166117831292, '00년생의': 11.960166117831292, '00년생이': 11.960166117831292, '00발라드': 11.043875385957136, '00세대': 11

---
### 코사인 유사도

In [88]:
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
cosine_similarity(tfid_matrix[0:1],tfid_matrix[1:2]) # 첫 번째 문장과 두 번째 문장 비교

array([[0.]])

---
### 유클리디안 유사도


- 가장 짧은 직선거리
- L1 정규화를 통해서 값을 제한한다
   - L1 정규화는 각 벡터 안의 요소 값을 모두 더한 것이 크기가 1이 되도록 벡터들의 크기를 조절하는 방법이다
- 유클리디안 유사도의 경우 값의 제한이 없기때문에 사용하기가 어려움 
- 그렇기 때문에 0과 1사이의 값을 갖도록 제한을 해줄건데 그 방법을 L1정규화를 통해 진행
- 보통 편의를 위해 정규화한 후 측정한다

In [92]:
from sklearn.metrics.pairwise import euclidean_distances

In [95]:
euclidean_distances(tfid_matrix[0:1],tfid_matrix[1:2])

array([[1.41421356]])

In [37]:
def l1_normalize(v):
    norm = np.sum(v)
    return v / norm

In [111]:
tfidf_norm_l1 = l1_normalize(tfid_matrix)
euclidean_distances(tfidf_norm_l1[0:1],tfidf_norm_l1[1:2])

array([[6.54076323e-06]])

---
### 맨해튼 유사도

- 가장 짧은 직각 이동거리
- 유클리디언 유사도와 같이 거리를 통해 유사도를 측정한다
  - 그렇기 때문에 L1 정규화를 통해 값을 제한한다

In [112]:
from sklearn.metrics.pairwise import manhattan_distances

In [113]:
manhattan_distances(tfidf_norm_l1[0:1],tfidf_norm_l1[1:2])

array([[1.20862066e-05]])

---
---
## 영어 텍스트 분류

In [39]:
import re
import pandas as pd
import numpy as np
import json

from konlpy.tag import Okt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [40]:
# 영어 데이터 전처리 과정을 함수로 정의
# 데이터 전처리 후 데이터를 하나의 리스트로 변경
# 출력값은 clean_train_txt

def english_preprocessing(txt, remove_stopwords = False): # 불용어 제거는 옵션으로 설정
    
#     html 용어 제거
#     txt = BeautifulSoup(txt, 'html5lib').get_text()
    
    # 영어 문자를 제외한 나머지 문자는 모두 공백으로 변환
    txt = re.sub("[^a-zA-Z0-9]",' ',txt)
    
    words = txt.lower().split() # 전부 소문자로 변환 후 단어마다 나눠서 단어리스트로 만든다
    
    # remove_stopwords == True인 경우 불용어 제거
    if remove_stopwords:
        
        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        words = [w for w in words if not w in stops] # stopwords에 있는 단어 제거
        
        clean_txt = ' '.join(words) # 각 단어들을 띄어쓰기로 묶어줌
    
    else:
        clean_txt = ' '.join(words)
    

    return clean_txt

In [95]:
clean_train_txt = []

text = train['plylst_title'].tolist()

clean_train_txt.append([english_preprocessing(txt, remove_stopwords=True) for txt in text])

In [96]:
english_title_token = clean_train_txt
english_title_token = pd.DataFrame(english_title_token)

In [127]:
english_title_token.T

,0
0,
1,
2,
3,
4,
...,...
115066,metal e sm 2
115067,edm
115068,1
115069,pop


In [98]:
english_title_token.T.to_csv('data/english_token.csv',index=False,encoding='utf-8')

In [145]:
# 위에서 나온 텍스트를 인덱스로 바꿔주고 
# tensorflow 전처리 모델에서 Tokenize 사용
# 정제된 데이터를 사용하고 인덱스로 구성된 벡터로 변환 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(playlist_title_token)
text_sequences = tokenizer.texts_to_sequences(playlist_title_token)

In [146]:
text_sequences
# 이제 값이 텍스트가 아닌 인덱스로 출력됨

[[134, 72, 4],
 [108, 90, 378, 395],
 [478, 423, 281, 116, 63, 8],
 [81, 53, 5, 2122, 636, 103, 114, 13],
 [67, 2, 1],
 [407, 34, 4136],
 [381, 511, 30, 71, 509, 11, 6, 244, 23, 8, 3],
 [10261, 28, 364, 82, 2372],
 [4910, 461, 180, 5263, 462],
 [227, 597, 4137, 895, 18050],
 [213, 18051],
 [1, 64, 7636, 17, 906, 6216, 1798, 43, 4365, 1005, 415, 2987, 83],
 [222, 90, 17, 93, 308, 374, 1999, 375, 12926],
 [169, 110, 537, 9, 19, 143, 123],
 [2564, 461, 168],
 [3196, 920, 32, 13, 14, 4],
 [36, 62, 594, 1, 3],
 [907, 498],
 [1015, 70, 286, 2319, 90, 82, 28, 137, 2988],
 [2714, 1485, 1057],
 [510, 741, 259, 32, 13, 479, 1209, 161],
 [],
 [67, 2, 6854, 17, 791, 144, 129, 61, 12],
 [41, 10, 1078],
 [222, 94, 274, 84, 7, 350, 87, 100, 56, 23, 85, 8],
 [65, 742, 194, 7, 727, 39, 17, 18052, 13, 1536],
 [41, 10, 386, 72, 349, 49, 3],
 [1032, 436, 498],
 [448, 14, 18053],
 [10262, 591, 334],
 [12927, 18, 12928, 192, 637, 319, 48],
 [44, 7, 921, 469, 277],
 [119, 7, 4366, 5264, 2, 1859, 1859],
 [128

In [151]:
# 위 처럼 변환한 경우 단어 사전이 필요함
word_vocab = tokenizer.word_index
word_vocab["<PAD>"] = 0

# 이건 한글을 통한 예시라서 신경 X 나중에 한글에 대해서 다시 할거임
# 불용어 제거도 안되어 있는 데이터임
word_vocab 

{'노래': 1,
 '의': 2,
 '들': 3,
 '음악': 4,
 '에': 5,
 '을': 6,
 '이': 7,
 '곡': 8,
 '한': 9,
 '좋은': 10,
 '감성': 11,
 '모음': 12,
 '때': 13,
 '듣는': 14,
 '가': 15,
 '듣기': 16,
 '를': 17,
 '에서': 18,
 '힙합': 19,
 '밤': 20,
 '발라드': 21,
 '날': 22,
 '하는': 23,
 '과': 24,
 '적': 25,
 '인': 26,
 '가을': 27,
 '내': 28,
 '리스트': 29,
 '사랑': 30,
 '팝': 31,
 '할': 32,
 '재즈': 33,
 'pop': 34,
 '와': 35,
 '나': 36,
 '카페': 37,
 '마음': 38,
 '위': 39,
 '는': 40,
 '기분': 41,
 '함께': 42,
 '1': 43,
 '겨울': 44,
 '2': 45,
 '봄': 46,
 '뉴에이지': 47,
 '클래식': 48,
 '명곡': 49,
 '팝송': 50,
 '잔잔한': 51,
 '플레이': 52,
 '분위기': 53,
 '들으면': 54,
 '신나는': 55,
 '추천': 56,
 '여름': 57,
 '으로': 58,
 '은': 59,
 '피아노': 60,
 'ost': 61,
 '만': 62,
 '있는': 63,
 '로': 64,
 '새벽': 65,
 '싶은': 66,
 '추억': 67,
 '연주': 68,
 '집': 69,
 '비': 70,
 '이별': 71,
 '같은': 72,
 '아이돌': 73,
 '듣고': 74,
 '대': 75,
 '어울리는': 76,
 '그': 77,
 '들어': 78,
 '되는': 79,
 '달달': 80,
 '크리스마스': 81,
 '도': 82,
 '뮤직': 83,
 '힐링': 84,
 '인디': 85,
 '드라이브': 86,
 '당신': 87,
 '혼자': 88,
 '오늘': 89,
 '너': 90,
 '멜로디': 91,
 '라디오': 92,
 '위로': 9

In [152]:
# 불용어 제거하면 더 줄어듬
print(len(word_vocab))

37734


In [153]:
data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)

data_configs

{'vocab': {'노래': 1,
  '의': 2,
  '들': 3,
  '음악': 4,
  '에': 5,
  '을': 6,
  '이': 7,
  '곡': 8,
  '한': 9,
  '좋은': 10,
  '감성': 11,
  '모음': 12,
  '때': 13,
  '듣는': 14,
  '가': 15,
  '듣기': 16,
  '를': 17,
  '에서': 18,
  '힙합': 19,
  '밤': 20,
  '발라드': 21,
  '날': 22,
  '하는': 23,
  '과': 24,
  '적': 25,
  '인': 26,
  '가을': 27,
  '내': 28,
  '리스트': 29,
  '사랑': 30,
  '팝': 31,
  '할': 32,
  '재즈': 33,
  'pop': 34,
  '와': 35,
  '나': 36,
  '카페': 37,
  '마음': 38,
  '위': 39,
  '는': 40,
  '기분': 41,
  '함께': 42,
  '1': 43,
  '겨울': 44,
  '2': 45,
  '봄': 46,
  '뉴에이지': 47,
  '클래식': 48,
  '명곡': 49,
  '팝송': 50,
  '잔잔한': 51,
  '플레이': 52,
  '분위기': 53,
  '들으면': 54,
  '신나는': 55,
  '추천': 56,
  '여름': 57,
  '으로': 58,
  '은': 59,
  '피아노': 60,
  'ost': 61,
  '만': 62,
  '있는': 63,
  '로': 64,
  '새벽': 65,
  '싶은': 66,
  '추억': 67,
  '연주': 68,
  '집': 69,
  '비': 70,
  '이별': 71,
  '같은': 72,
  '아이돌': 73,
  '듣고': 74,
  '대': 75,
  '어울리는': 76,
  '그': 77,
  '들어': 78,
  '되는': 79,
  '달달': 80,
  '크리스마스': 81,
  '도': 82,
  '뮤직': 83,
  '힐링': 84,
  '인디'

In [154]:
# 현재 데이터는 서로 길이가 다르기 때문에 데이터의 길이를 하나로 통일해야 모델에 적용할 수 있음
# 특정 길이를 최대 길이로 정하고 더 긴 데이터의 경우 0 값을로 패딩하는 작업 진행

MAX_SEQUENCE_LENGTH = 174
# 여기서 174는 단어 개수 통계에서의 중앙값

train_input = pad_sequences(text_sequences, MAX_SEQUENCE_LENGTH, padding='post')

print('Shape : ', train_input.shape)

Shape :  (115071, 174)


---
---
---
### 한극 텍스트 분류

In [70]:
train['plylst_title'][:10]

0                              여행같은 음악
1                              요즘 너 말야
2                편하게, 잔잔하게 들을 수 있는 곡.-
3                크리스마스 분위기에 흠뻑 취하고 싶을때
4                             추억의 노래 ㅋ
5                       2017 Pop Trend
6    짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!
7                        멍청이.. 내맘도 몰라.
8        DANCING IN THE MOON-LIGHT .01
9                [록/메탈] Written by 이일우
Name: plylst_title, dtype: object

In [5]:
okt = Okt()

def korean_preprocessing(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트

#     html 용어 제거
#     txt = BeautifulSoup(txt, 'html5lib').get_text()
        
    
    txt = re.sub('[^가-힣0-9]',"",txt)
    # 한글과 숫자만 나오게 함
    
    
    # okt 객체를 활용해 형태소 단위로 나눈다
    word_txt = okt.morphs(txt, stem = True)
    
    if removes_stopwords == True:
        word_txt = [token for token in word_txt if not token in stop_words]    
    
    return word_txt

In [80]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()
stopword

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를',
 '들면',
 '들자면',
 '저',
 '소인',
 '소생',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수',
 '없다',
 '해서는',
 '안된다',
 '뿐만',
 '아니라',
 '만이',
 '아니다',
 '만은',
 '막론하고',
 '관계없이',
 '그치지',
 '않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지',
 '따지지',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만',
 '못하다',
 '하는',
 '편이',
 '낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이',
 '외에',
 '밖에',
 '하여야',
 '비로소',
 '한다면',
 '몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할',
 '생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게',
 '함으로써',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에',
 '틀림없다',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간'

In [12]:
clean_train_txt = []

okt = Okt()

text = train['plylst_title'].tolist()

x = [korean_preprocessing(txt, okt, removes_stopwords= True, stop_words=stopword) for txt in text]
clean_train_txt.append(x)

In [67]:
clean_train_txt.append(x)

In [68]:
korean_title_token = pd.DataFrame(clean_train_txt).T
korean_title_token.head(10)

,0,1
0,,"[여행, 음악]"
1,,[요즘]
2,,"[편하다, 잔잔하다, 들다, 곡]"
3,,"[크리스마스, 분위기, 흠뻑, 취하, 고, 싶다]"
4,,"[추억, 노래]"
5,2017 pop trend,[2017]
6,,"[짝사랑, 고, 백, 사랑, 이별, 슬픔, 감성, 자극, 는, 곡]"
7,,"[멍청이, 맘, 도, 모르다]"
8,dancing moon light 01,[01]
9,written,"[록, 메탈, 이일우]"


In [73]:
korean_title_token.to_csv('data/korean_token.csv',index=False,encoding='utf-8')

In [ ]:
english_preprocessing(train['playlist_title_txt'])

---
---
---
### 한글 영어 같이 전처리

In [29]:
def total_preprocessing(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트
    
    
    txt = re.sub('[^A-Za-z가-힣0-9]'," ",txt)
        
    
    # okt 객체를 활용해 형태소 단위로 나눈다
    word_txt = okt.morphs(txt, stem = True)
    
    if removes_stopwords == True:
        word_txt = [token for token in word_txt if not token in stop_words]    
    
    return word_txt 

In [84]:
okt = Okt()

text = train['plylst_title']

kr_en_token = []

x = [total_preprocessing(txt,okt,removes_stopwords=True, stop_words=stopword) for txt in text]

kr_en_token.append(x)
kr_en_token

[[['여행', '음악'],
  ['요즘'],
  ['편하다', '잔잔하다', '들다', '곡'],
  ['크리스마스', '분위기', '흠뻑', '취하', '고', '싶다'],
  ['추억', '노래'],
  ['2017', 'Pop', 'Trend'],
  ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
  ['멍청이', '맘', '도', '모르다'],
  ['DANCING', 'IN', 'THE', 'MOON', 'LIGHT', '01'],
  ['록', '메탈', 'Written', 'by', '이일우'],
  ['걸그룹', '땐쓰쏭'],
  ['노래', '의지', '불', '태우다', '1일', '1', '다짐', 'st', '용', '프로필', '뮤직'],
  ['지치다', '위로', '해주다', '제목', '편지', '준비', '해봤다'],
  ['트렌디', '그루브', '힙합', '알앤비', 'MUSIC'],
  ['Autumn', 'in', 'Jazz'],
  ['걍', '게임', '듣다', '음악'],
  ['알다', '노래'],
  ['축가', '듀엣'],
  ['비', '내리다', '날다', '도'],
  ['올림픽', '스밍', '목록'],
  ['조용하다', '맥주', '한잔', '새벽', '감성'],
  [],
  ['추억', '명화', '만나다', '영화', 'OST', '모음'],
  ['기분', '좋다', '여름날'],
  ['지치다', '하루', '끝', '힐링', '필요하다', '추천', '이다', '곡'],
  ['새벽', '찬', '바람', '침대', '위', '감싸다', '국외'],
  ['기분', '좋다', '햇살', '숨기다', '명곡'],
  ['사랑스럽다', '남녀', '듀엣'],
  ['즐기다', '듣다', '소래'],
  ['닥힙추', '8월', 'Playlist'],
  ['거슈윈', '스타워즈', '미국', '대표', '클래식'],
  ['겨울', '기다리다', '미

In [87]:
pd.DataFrame(kr_en_token).T.to_csv('data/kr_en_token.csv', index=False, encoding='utf-8')

In [128]:
okt.morphs('고백')

['고백']

In [7]:
txt = train['plylst_title'].tolist()

In [11]:
txt = []

for i, document in enumerate(train['plylst_title']):
    document = re.sub(r'[^A-Za-z]', '', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip())

In [12]:
txt

['',
 '',
 '',
 '',
 '',
 'PopTrend',
 '',
 '',
 'DANCINGINTHEMOONLIGHT',
 'Writtenby',
 '',
 'st',
 '',
 'MUSIC',
 'AutumninJazz',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'OST',
 '',
 '',
 '',
 '',
 '',
 '',
 'Playlist',
 '',
 '',
 '',
 '',
 'POP',
 '',
 '',
 '',
 '',
 '',
 '',
 'CafePopMusic',
 '',
 'Jazz',
 '',
 '',
 'ROCK',
 '',
 'ElectronicaloungePart',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'DJLimeVol',
 '',
 '',
 '',
 'MUSIC',
 '',
 'ROCK',
 '',
 '',
 '',
 '',
 '',
 '',
 'EDM',
 'Mikewillmadeit',
 '',
 '',
 '',
 'RB',
 '',
 '',
 'LasseLindh',
 'JarreauVandal',
 '',
 '',
 '',
 '',
 '',
 'IndieRockPops',
 '',
 '',
 '',
 '',
 'ItaliaCafe',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'DJNCTDREAM',
 '',
 '',
 'TAKEALOOK',
 '',
 '',
 'Popincafe',
 '',
 '',
 '',
 '',
 'MTV',
 '',
 '',
 '',
 'EDM',
 'OST',
 '',
 '',
 'Dance',
 'WEEKLYCHOICE',
 '',
 'BGMPopVol',
 '',
 '',
 '',
 '',
 '',
 'JAZZ',
 '',
 '',
 'Katyperry',
 'JAZZYMOOD',
 '',
 '',
 '',
 '',
 '',
 '',
 'JAZZ',
 '',
 

In [11]:
x = okt.phrases('국힙 힙합갬성 흥얼흥얼 쇼미 외힙 기리보이의노래는길이보인다 왜이리딱딱한것이냐',)

In [12]:
x = okt.phrases('국힙 힙합갬성 흥얼흥얼 쇼미 외힙 기리보이의노래는길이보인다 왜이리딱딱한것이냐')
y = [okt.morphs(txt) for txt in x]
y

[['국힙'],
 ['국힙', '힙합'],
 ['성', '흥얼흥얼'],
 ['성', '흥얼흥얼', '쇼'],
 ['외힙'],
 ['외힙', '기리보이'],
 ['외힙', '기리보이', '의', '노래는길', '이보', '인'],
 ['힙합'],
 ['흥얼흥얼'],
 ['기리보이'],
 ['이보', '인']]

In [20]:
x = okt.morphs('ㅅ')
y = [okt.phrases(txt) for txt in x]
y

[[], []]

In [22]:
x

['7', '살']

In [23]:
from pykospacing import spacing

ImportError: cannot import name 'spacing' from 'pykospacing' (C:\Users\medici\anaconda3\lib\site-packages\pykospacing\__init__.py)